<a href="https://colab.research.google.com/github/Latindude101/Quantitative-Economics/blob/main/QE_PS6_Q6_(R).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup and OLS

In [ ]:
install.packages('estimatr')
install.packages('car')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘Formula’, ‘RcppEigen’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘matrixStats’, ‘RcppArmadillo’, ‘numDeriv’, ‘SparseM’, ‘MatrixModels’, ‘conquer’, ‘sp’, ‘data.table’, ‘openxlsx’, ‘minqa’, ‘nloptr’, ‘statmod’, ‘carData’, ‘abind’, ‘pbkrtest’, ‘quantreg’, ‘maptools’, ‘rio’, ‘lme4’




In [ ]:
fert <- read.csv('fertility.csv')
head(fert)
library(estimatr)
library(car)

In [20]:
# Extract values
morekids <- fert$morekids
boy1st <- fert$boy1st
boy2nd <- fert$boy2nd
samesex <- fert$samesex
agem1 <- fert$agem1
black <- fert$black
hispan <- fert$hispan
othrace <- fert$othrace
weeksm1 <- fert$weeksm1


In [5]:
# a)
# OLS regression: Y = B0 + B1Di + U
# morekids = 1 if mum has more than 2 kids
add_kids <- summary(lm_robust(formula = weeksm1 ~ morekids, data = fert))
add_kids
ad_coef <- add_kids$coefficient["morekids","Estimate"]
ad_err <- add_kids$coefficient["morekids","Std. Error"]
ad_err
# Using sprintf() for string interpolation
sprintf("On average, in the sample, the effect of having an additional child when a woman has already had two children on weeks spent working holding other variables constant is %s weeks.", round(ad_coef, digits = 1))


Call:
lm_robust(formula = weeksm1 ~ morekids, data = fert)

Standard error type:  HC2 

Coefficients:
            Estimate Std. Error t value Pr(>|t|) CI Lower CI Upper     DF
(Intercept)   21.068    0.05607  375.77        0   20.959   21.178 254652
morekids      -5.387    0.08715  -61.81        0   -5.558   -5.216 254652

Multiple R-squared:  0.01431 ,	Adjusted R-squared:  0.0143 
F-statistic:  3821 on 1 and 254652 DF,  p-value: < 2.2e-16

[1] 0.08714918

[1] "On average, in the sample, the effect of having an additional child when a woman has already had two children on weeks spent working holding other variables constant is -5.4 weeks."

In [6]:
# b)
"The regression in a) will not give an estimate of the causal effect of fertility on labour supply because the residual of the regression is not orthogonal to variables used to estimate the labour supply."
"For example, there is likely to be a correlation between number of children and wealth (not included as a variable in the regression) which also affects number of weeks worked."
"Or women who want to focus on their career may have fewer children (unobserved preferences)."
"Perhaps reverse causality."

[1] "The regression in a) will not give an estimate of the causal effect of fertility on labour supply because the residual of the regression is not orthogonal to variables used to estimate the labour supply."

[1] "For example, there is likely to be a correlation between number of children and wealth (not included as a variable in the regression) which also affects number of weeks worked."

[1] "Or women who want to focus on their career may have fewer children (unobserved preferences)."

[1] "Perhaps reverse causality."

In [ ]:
# c)
# i) t test of mean morekids between boy1st==1 and boy1st==0
m_morekids <- fert$morekids[fert$boy1st==1]
f_morekids <- fert$morekids[fert$boy1st==0]
mean(m_morekids)
mean(f_morekids)

# 1. Hypotheses:
"H0: No diff in mean fertility between those whose first child was male vs female"
"ha: There is a diff in mean fertility between the groups"

# 2. Decision Rule:
"alpha = 0.05"
crit <- qt(0.975, nrow(fert))
sprintf("Reject H0 if |t| > %s", qt(0.975, nrow(fert)))

# 3. Calculate t value:
ttest <- t.test(f_morekids, m_morekids, mu=0, alternative="two.sided")
ttest

# 4. Compare with critical value:
sprintf("%s > %s", ttest$statistic, crit)
"We can therefore reject the hypothesis that there is no difference in fertility between those who have male first child and those who have a female first child."
'Those who have a male first child on average in the same go on to have more than two children less frequently than those who have a female first child.'
"This could be because of the desire to have a son that could mean that those who have a male first child are less likely to have more than two kids that those whose first two children are female."

[1] 0.376275

[1] 0.3851055

[1] "H0: No diff in mean fertility between those whose first child was male vs female"

[1] "ha: There is a diff in mean fertility between the groups"

[1] "alpha = 0.05"

...


	Welch Two Sample t-test

data:  f_morekids and m_morekids
t = 4.5867, df = 253679, p-value = 4.505e-06
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 0.005057134 0.012603982
sample estimates:
mean of x mean of y 
0.3851055 0.3762750 


[1] "4.5867253687037 > 1.95997330024809"

[1] "We can therefore reject the hypothesis that there is no difference in fertility between those who have male first child and those who have a female first child."

[1] "Those who have a male first child on average in the same go on to have more than two children less frequently than those who have a female first child."

[1] "This could be because of the desire to have a son that could mean that those who have a male first child are less likely to have more than two kids that those whose first two children are female."

In [8]:
# c) alternatively different effects can be estimated using regressions and testing whether the coefficient is zero
lm_robust(morekids ~ boy1st, data = fert)
# gives same t value as mean comparison
"Estimate of coefficient on boy1st of -0.0088 is statistically significant but very small ('significant' does not mean large in statistics: significant means large relative to its imprecision)"

                Estimate  Std. Error    t value     Pr(>|t|)    CI Lower
(Intercept)  0.385105523 0.001383757 278.304355 0.000000e+00  0.38239340
boy1st      -0.008830558 0.001925242  -4.586725 4.504663e-06 -0.01260398
                CI Upper     DF
(Intercept)  0.387817649 254652
boy1st      -0.005057134 254652

In [9]:
# ii) t test of the mean of morekids between samesex==1 and samesex==0
# samesex==1 when first two children are of the same sex
same_morekids <- fert$morekids[fert$samesex==1]
diff_morekids <- fert$morekids[fert$samesex==0]
mean(same_morekids)
mean(diff_morekids)

# 1. Hypotheses:
"H0: No difference in average fertility between women whose first two children are of the same sex and women whose first two children are of different sex"
"Ha: the average fertilities are not equal"

# 2. Decision Rule:
"alpha = 0.05"
"Reject if t value lies outside +/- 1.959973 (critical values with 254614 d.o.f.)"
# calculate critical t values
nrow(fert)

# inputting 0.975 to qt() gives the critical value at the 0.05% s.l. for a two sided test
# qt() is TDist quantile function
qt(.975, nrow(fert))
# WHY IS MY COUNT OF DOF DIFFERENT FROM R'S T TEST (off by 40)

# 3. Calculate:
ttest <- t.test(same_morekids, diff_morekids, mu=0, alternative="two.sided")
ttest

# 4. Compare critical values
" t value: 35.188 > 1.959973"
sprintf("We can therefore reject the null hypothesis that there is no difference between the average fertility of women whose first two children are of the same sex and women whose first two children are of different sex")
"This could also be due to the desire for a male heir that means that those with two female first children (samesex==1) are more likely to have further children."

[1] 0.4139501

[1] 0.3464248

[1] "H0: No difference in average fertility between women whose first two children are of the same sex and women whose first two children are of different sex"

[1] "Ha: the average fertilities are not equal"

[1] "alpha = 0.05"

...

[1] 1.959973


	Welch Two Sample t-test

data:  same_morekids and diff_morekids
t = 35.188, df = 254614, p-value < 2.2e-16
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 0.06376406 0.07128646
sample estimates:
mean of x mean of y 
0.4139501 0.3464248 


[1] " t value: 35.188 > 1.959973"

[1] "We can therefore reject the null hypothesis that there is no difference between the average fertility of women whose first two children are of the same sex and women whose first two children are of different sex"

[1] "This could also be due to the desire for a male heir that means that those with two female first children (samesex==1) are more likely to have further children."

In [10]:
# d)
'samesex might not be a valid instrument for morekids if it does not meet the exogeneity condition'
"Three conditions for a valid instrument: exclusion, relevance, exogeneity"
'samesex is exogenously assigned (assigned as if random) (theoretically) but may not only affect labour supply through morekids:'
"exogeneity could be tested with an F test of the other variables determining samesex"
"c): samesex is revelant to the determination of morekids"
'It could be that having children of different sexes means more work for the mother, e.g. ferrying them to different activities, resulting in correlation between samesex and weeksm1'
"This would be an effect of samesex on labour supply that does not go through morekids"

# Regression d
exog_reg <- lm_robust(formula = weeksm1 ~ samesex, data=fert)

# F test of significance
# hypothesis matrix requires single =

# 1. Hypotheses:
# H0: OLS coefficient on samesex = 0
# Ha: OLS coefficient on samesex != 0

# 2. Decision Rule:
# Reject H0 if F statistic > 10

# 3. Calculate F statistic
ftest <- linearHypothesis(exog_reg, c("samesex=0"), test="F")
ftest

# 4. Compare F statistic to critical value:
sprintf("%s > %s", ftest$F[2], 10)
"Therefore we can reject the null hypothesis"


[1] "samesex might not be a valid instrument for morekids if it does not meet the exogeneity condition"

[1] "samesex is exogenously assigned (assigned as if random) but may not only affect labour supply through morekids:"

[1] "It could be that having children of different sexes means more work for the mother, e.g. ferrying them to different activities, resulting in correlation between samesex and weeksm1"

,Res.Df,Df,F,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>
1,254653,NA,NA,NA
2,254652,1,24.1957,8.708031e-07


[1] "24.1957035172991 > 10"

[1] "Therefore we can reject the null hypothesis"

# 2SLS

In [11]:
# e)
# 2SLS manually
# Model: y = B0 + B1Xi + Ui
# weeksm1 ~ morekids
# Regress morekids on the instrument to calculate fitted values
fsls <- lm_robust(formula = morekids ~ samesex, data = fert)
summary(fsls)
# how are fitted values different from morekids?
morekids_hat = fitted.values(fsls)
tsls <- lm_robust(formula = weeksm1 ~ morekids_hat, data = fert)
summary(tsls)



Call:
lm_robust(formula = morekids ~ samesex, data = fert)

Standard error type:  HC2 

Coefficients:
            Estimate Std. Error t value   Pr(>|t|) CI Lower CI Upper     DF
(Intercept)  0.34642   0.001341  258.34  0.000e+00  0.34380  0.34905 254652
samesex      0.06753   0.001919   35.19 1.388e-270  0.06376  0.07129 254652

Multiple R-squared:  0.004835 ,	Adjusted R-squared:  0.004831 
F-statistic:  1238 on 1 and 254652 DF,  p-value: < 2.2e-16


Call:
lm_robust(formula = weeksm1 ~ morekids_hat, data = fert)

Standard error type:  HC2 

Coefficients:
             Estimate Std. Error t value  Pr(>|t|) CI Lower CI Upper     DF
(Intercept)    21.421     0.4906  43.663 0.000e+00   20.460   22.383 254652
morekids_hat   -6.314     1.2836  -4.919 8.708e-07   -8.829   -3.798 254652

Multiple R-squared:  9.502e-05 ,	Adjusted R-squared:  9.109e-05 
F-statistic:  24.2 on 1 and 254652 DF,  p-value: 8.708e-07

In [17]:
# e) using packages
ivreg <- iv_robust(formula = weeksm1 ~ morekids | samesex, diagnostics=TRUE)
iv_sum <- summary(ivreg)
iv_sum
sprintf('SE of 2SLS (%s) >> SE of OLS (%s): samesex is not a valid instrument as explained in d)', iv_sum$coefficient["morekids", "Std. Error"], ad_err)
'Estimate of -6.3 from 2SLS vs -5.3 from OLS:'


Call:
iv_robust(formula = weeksm1 ~ morekids | samesex, diagnostics = TRUE)

Standard error type:  HC2 

Coefficients:
            Estimate Std. Error t value  Pr(>|t|) CI Lower CI Upper     DF
(Intercept)   21.421     0.4873  43.963 0.000e+00   20.466   22.376 254652
morekids      -6.314     1.2747  -4.953 7.308e-07   -8.812   -3.815 254652

Multiple R-squared:  0.01388 ,	Adjusted R-squared:  0.01388 
F-statistic: 24.53 on 1 and 254652 DF,  p-value: 7.308e-07

Diagnostics:
                  numdf  dendf    value p.value    
Weak instruments      1 254652 1238.171  <2e-16 ***
Wu-Hausman            1 254651    0.531   0.466    
Overidentifying       0     NA       NA      NA    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

[1] "SE of 2SLS (1.27468565210554) >> SE of OLS (0.0871491793698437): samesex is not a valid instrument as explained in d)"

[1] "Estimate of -6.3 from 2SLS vs -5.3 from OLS:"

In [21]:
# f) manually
# othrace does not include white so no perfect multicollinearity with black, hispan, and othrace
# missed out age!!!
sum(fert$black)
sum(fert$othrace)
sum(fert$hispan)
fsls <- lm_robust(formula = morekids ~ samesex + black + hispan + othrace, data = fert)
summary(fsls)
morekids_hat <- fitted.values(fsls)
tsls <- lm_robust(formula = weeksm1 ~ morekids_hat, data = fert)
summary(tsls)

[1] 13156

[1] 14348

[1] 18897


Call:
lm_robust(formula = morekids ~ samesex + black + hispan + othrace, 
    data = fert)

Standard error type:  HC2 

Coefficients:
            Estimate Std. Error t value   Pr(>|t|) CI Lower CI Upper     DF
(Intercept)  0.32986   0.001383 238.447  0.000e+00  0.32715  0.33257 254649
samesex      0.06767   0.001911  35.404 6.935e-274  0.06392  0.07141 254649
black        0.09224   0.004459  20.688  5.325e-95  0.08350  0.10098 254649
hispan       0.13836   0.004147  33.368 1.334e-243  0.13024  0.14649 254649
othrace      0.02597   0.004655   5.578  2.433e-08  0.01684  0.03509 254649

Multiple R-squared:  0.01272 ,	Adjusted R-squared:  0.01271 
F-statistic: 802.5 on 4 and 254649 DF,  p-value: < 2.2e-16


Call:
lm_robust(formula = weeksm1 ~ morekids_hat, data = fert)

Standard error type:  HC2 

Coefficients:
             Estimate Std. Error t value  Pr(>|t|) CI Lower CI Upper     DF
(Intercept)     14.76     0.3062   48.20 0.000e+00   14.161    15.36 254652
morekids_hat    11.19     0.7985   14.01 1.408e-44    9.622    12.75 254652

Multiple R-squared:  0.000785 ,	Adjusted R-squared:  0.0007811 
F-statistic: 196.3 on 1 and 254652 DF,  p-value: < 2.2e-16

In [22]:
# f)
ivreg <- iv_robust(formula = weeksm1 ~ morekids | samesex + agem1 + black + hispan + othrace, diagnostics = TRUE)
summary (ivreg)
'The coefficient on morekids is now positive.'
'Standard error is reduced as the inclusion of more variables allows for more precise estimation.'


Call:
iv_robust(formula = weeksm1 ~ morekids | samesex + agem1 + black + 
    hispan + othrace, diagnostics = TRUE)

Standard error type:  HC2 

Coefficients:
            Estimate Std. Error t value   Pr(>|t|) CI Lower CI Upper     DF
(Intercept)    8.084     0.2714   29.79 1.115e-194    7.552    8.616 254652
morekids      28.731     0.7086   40.55  0.000e+00   27.342   30.120 254652

Multiple R-squared:  -0.5596 ,	Adjusted R-squared:  -0.5596 
F-statistic:  1644 on 1 and 254652 DF,  p-value: < 2.2e-16

Diagnostics:
                  numdf  dendf value p.value    
Weak instruments      5 254648  1304  <2e-16 ***
Wu-Hausman            2 254650  2540  <2e-16 ***
Overidentifying       4     NA  2326  <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

[1] "The coefficient on morekids is now positive."

[1] "Standard error is reduced as the inclusion of more variables allows for more precise estimation."

In [24]:
iv_robust(weeksm1 ~ morekids + agem1 + black + hispan + othrace | samesex  + agem1 + black + hispan + othrace, data = fert, diagnostics = TRUE)


              Estimate Std. Error    t value      Pr(>|t|)   CI Lower
(Intercept) -4.7918935 0.38979270 -12.293441  1.004902e-34 -5.5558768
morekids    -5.8210509 1.24640073  -4.670288  3.009303e-06 -8.2639631
agem1        0.8315975 0.02264086  36.729935 1.446269e-294  0.7872220
black       11.6232731 0.23180281  50.142934  0.000000e+00 11.1689458
hispan       0.4041802 0.26080282   1.549754  1.212018e-01 -0.1069863
othrace      2.1309620 0.21099514  10.099579  5.606068e-24  1.7174172
              CI Upper     DF
(Intercept) -4.0279102 254648
morekids    -3.3781388 254648
agem1        0.8759730 254648
black       12.0776004 254648
hispan       0.9153468 254648
othrace      2.5445068 254648

In [ ]:
iv_robust(weeksm1 ~ morekids + agem1 + black + hispan + othrace | samesex  + agem1 + black + hispan + othrace, data = fertility, diagnostics = TRUE)


In [25]:
# g)
# perfect multicollinearity of samesex and two_male and two_female?
# Two_male: is the first child a girl? If girl, then 0. If boy, then if samesex, then 1. If boy but not samesex, then 0.
two_male <- ifelse((fert$boy1st==0), 0, ifelse((fert$samesex==1), 1, 0))
two_female <- ifelse((fert$boy1st==1), 0, ifelse((fert$samesex==1), 1, 0))
two_male
two_female

# adding/excluding samesex does not change anything
ivreg <- iv_robust(formula = weeksm1 ~ morekids | samesex + black + hispan + othrace + two_male + two_female, diagnostics=TRUE)
summary (ivreg)


ivreg <- iv_robust(formula = weeksm1 ~ morekids | two_male + two_female, diagnostics=TRUE)
summary (ivreg)
'The regression with just two_male and two_female as instruments, unsurprisingly, gives very similar estimates to the regression using samesex as an instrument.'

[1] 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 1 1 1 0 1 0 1 1 0 0 0 0 0 0 0 0
   [37] 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1
   [73] 1 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 1
  [109] 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 1 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1 0 1 0
  [145] 0 0 0 0 0 1 0 0 0 1 0 1 0 1 0 0 1 1 0 0 1 0 0 0 0 0 0 1 0 1 0 1 0 0 0 0
  [181] 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0
  [217] 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0
  [253] 0 0 0 1 0 0 0 0 0 0 0 0 1 1 1 1 0 1 0 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0
  [289] 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0
  [325] 0 1 0 0 0 1 0 0 1 0 1 1 0 1 1 0 0 0 0 1 0 1 0 0 0 1 0 1 0 1 0 0 0 0 0 0
  [361] 0 1 0 0 0 0 0 1 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0
  [397] 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 1 1 0 0 1 0 0 0 0 1 1
  [433] 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 0 0 1 0 0 0 0 1 0 1 1 0 1 0 0 1
  [469] 0 1 0 1 0 1 0 1 0 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0
  [505] 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 1 0 1 0 1 1 1 0 0 0 0 1 0 1 0 1
  [541] 1 0 1 0 1 0 1 0 0 0 0 0 1 0 0 1 0 0 1 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 1 0
  [577] 0 0 0 0 1 1 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 1 0 1 0 0 1 1 0 1 0 1 0 1 1 0
  [613] 0 0 0 0 1 0 1 0 0 1 0 0 1 0 1 0 1 1 0 1 0 0 0 0 1 0 0 0 1 1 0 0 0 1 0 0
  [649] 1 0 0 0 0 0 0 1 0 0 1 0 0 1 0 1 0 0 1 0 0 1 0 0 1 0 1 0 0 0 0 0 0 1 1 0
  [685] 0 1 0 0 0 1 0 0 0 0 1 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 1 0
  [721] 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 1 0 1 1 1 0 0 0 0 0
  [757] 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 1 1 0 0
  [793] 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0
  [829] 0 1 1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 1 0 1 0 0 0 1 0 0 0
  [865] 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0
  [901] 0 0 0 1 0 0 0 1 1 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1
  [937] 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 1 0 1 0 1 0 0 1
  [973] 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 1 0 0 0 0 0 1 0 1 1 0
 [1009] 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1
 [1045] 0 0 1 0 0 0 1 0 0 0 1 1 1 1 0 0 1 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 [1081] 0 0 0 0 0 0 0 0 1 0 1 1 1 0 0 1 0 1 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1
 [1117] 0 1 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 1 0 1 0 1 0 1 0 0 0
 [1153] 0 0 0 0 1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1
 [1189] 0 1 0 0 1 0 0 1 0 0 1 1 0 1 1 1 0 1 0 1 1 0 1 1 0 0 0 0 0 1 0 1 0 0 1 0
 [1225] 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0
 [1261] 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 1 1 0 1 0 0 0 1 1 0
 [1297] 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 1 0 1 0 1 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0
 [1333] 0 1 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 1 1 0 1 0 1 0 1 0 1 0 0 0 0
 [1369] 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 1
 [1405] 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 [1441] 0 1 1 1 1 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1
 [1477] 0 1 1 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 1 0 1 1 0 0 0 0 1
 [1513] 0 0 0 1 0 0 1 1 1 0 0 0 0 1 0 0 1 1 1 1 1 0 1 0 1 0 0 0 0 0 1 1 0 0 0 0
 [1549] 0 1 1 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1
 [1585] 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 1 0 0 0 0 0 1 0 1 0 0 0 1 0 0
 [1621] 1 1 1 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 1 0 1 0 0 0 0 1 0 1 0 1 0 0 0 0 1 1
 [1657] 0 1 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 1
 [1693] 1 0 0 1 1 0 1 0 1 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 1 0 0
 [1729] 0 0 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0
 [1765] 0 1 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 1 1
 [18

[1] 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0
   [37] 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0
   [73] 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 1 1 0 0 0 0 1 0 1 0 0 0 0 0 1 0
  [109] 0 0 1 0 0 1 1 1 0 0 1 0 0 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
  [145] 0 0 0 0 0 0 1 1 0 0 1 0 0 0 0 1 0 0 1 1 0 1 0 0 0 1 1 0 0 0 0 0 1 0 0 0
  [181] 0 0 0 1 0 1 0 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 1 1 0 1 0
  [217] 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 1 0 0 0 0 1 1 0
  [253] 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 0 1 0 1 0 1 1 1
  [289] 0 0 0 1 0 0 0 0 0 1 0 1 1 1 0 0 1 0 0 0 0 0 1 0 0 0 0 1 1 0 1 0 1 1 0 0
  [325] 1 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 1 0 1
  [361] 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 1 1 0 0 0 1 1 1 0 0 0 0 0 0 0 0
  [397] 1 0 1 0 0 1 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0
  [433] 1 0 0 0 1 1 1 1 1 0 0 1 1 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0
  [469] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 1 0
  [505] 0 0 0 0 0 1 0 1 1 0 0 0 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0
  [541] 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 1
  [577] 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
  [613] 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0
  [649] 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 1 0 0 1 0 1 0 0 1 1 1 0 0 1
  [685] 0 0 1 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 1 1 0 0 0 0 0
  [721] 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 1 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 1 0
  [757] 0 0 0 0 0 1 1 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0
  [793] 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0
  [829] 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 1 0
  [865] 0 0 0 1 0 0 1 0 1 0 0 1 0 0 0 1 1 0 0 1 0 0 0 0 1 1 1 0 0 1 0 0 0 1 0 0
  [901] 1 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 1 0 1 0 1 1 1 0 1 0 0 0 0
  [937] 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0
  [973] 0 0 0 0 0 0 0 1 0 0 0 0 1 1 1 1 1 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1
 [1009] 1 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 [1045] 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 1 0 0
 [1081] 1 0 0 1 1 0 0 1 0 1 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0 1 1 0 0 1 1 1 0 0 0
 [1117] 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 1 0 0 0 0 0 0 0 1 0 1 0 0
 [1153] 0 0 1 0 0 0 0 0 0 0 1 0 1 1 1 0 0 1 1 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0
 [1189] 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 1 0 0
 [1225] 1 0 0 0 1 0 1 1 1 0 0 0 0 0 0 0 0 0 1 1 1 0 0 1 0 1 0 0 0 1 0 0 0 1 0 1
 [1261] 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 1
 [1297] 0 0 0 0 1 0 0 1 0 1 0 0 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 1 0 1 0 0 1 1
 [1333] 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
 [1369] 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0
 [1405] 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 0 1
 [1441] 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0
 [1477] 0 0 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0
 [1513] 1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 1 1 0 1
 [1549] 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 1 0 0 0 1 1 0 1 0 0 0 0 0 0 1 0
 [1585] 0 0 1 0 1 0 1 1 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0
 [1621] 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 [1657] 0 0 0 0 1 0 1 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 1 1 0
 [1693] 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0
 [1729] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 1 0 0 1 0 0 0 0 1 0 0 1 0 0
 [1765] 0 0 1 0 0 1 0 0 1 0 1 1 0 1 0 0 0 0 1 0 1 1 0 0 1 1 0 0 1 0 1 0 0 0 0 0
 [18


Call:
iv_robust(formula = weeksm1 ~ morekids | samesex + black + hispan + 
    othrace + two_male + two_female, diagnostics = TRUE)

Standard error type:  HC2 

Coefficients:
            Estimate Std. Error t value  Pr(>|t|) CI Lower CI Upper     DF
(Intercept)    14.81     0.3223   45.96 0.000e+00   14.182    15.45 254652
morekids       11.03     0.8393   13.14 2.038e-39    9.383    12.67 254652

Multiple R-squared:  -0.1185 ,	Adjusted R-squared:  -0.1185 
F-statistic: 172.6 on 1 and 254652 DF,  p-value: < 2.2e-16

Diagnostics:
                  numdf  dendf  value p.value    
Weak instruments      6 254647     NA      NA    
Wu-Hausman            2 254650  222.3  <2e-16 ***
Overidentifying       5     NA 2149.5  <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


Call:
iv_robust(formula = weeksm1 ~ morekids | two_male + two_female, 
    diagnostics = TRUE)

Standard error type:  HC2 

Coefficients:
            Estimate Std. Error t value  Pr(>|t|) CI Lower CI Upper     DF
(Intercept)   21.267     0.4763  44.650 0.000e+00    20.33   22.200 254652
morekids      -5.908     1.2458  -4.742 2.117e-06    -8.35   -3.466 254652

Multiple R-squared:  0.01417 ,	Adjusted R-squared:  0.01417 
F-statistic: 22.49 on 1 and 254652 DF,  p-value: 2.117e-06

Diagnostics:
                  numdf  dendf   value p.value    
Weak instruments      2 254651 645.871  <2e-16 ***
Wu-Hausman            1 254651   0.176   0.675    
Overidentifying       1     NA   2.282   0.131    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

[1] "The regression with just two_male and two_female as instruments, unsurprisingly, gives very similar estimates to the regression using samesex as an instrument."

In [ ]:
two_boys  = as.numeric( fertility$samesex & fertility$boy1st )
two_girls  = as.numeric( fertility$samesex & !fertility$boy1st )
# Peter's dummies use logical operations on dummy values converting them to Booleans

In [ ]:
# Peter's iv_robust() with diff variables names
iv_robust(weeksm1 ~ morekids + agem1 + black + hispan + othrace | two_girls + two_boys + agem1 + black + hispan + othrace, data = fertility, diagnostics = TRUE)


In [ ]:
# h)
'p value of 0.131 in Overidentifying restrictions means that there is not sufficient evidence at the 5% significance level to reject the null of exogeneity for two_male and two_female as instrumental variables'
'You do not want to reject exogeneity'